# Taller 1 Cadenas de Markov



**Libraries**
-

In [143]:
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 

# **1. Carga y Preprocesamiento de Datos**

## 1.1 Carga de Base de Datos

In [144]:
df = pd.read_excel("Base_de_Datos_Divisas_Banco_de_la_Republica.xlsx")

## 1.2 Análisis Exploratorio
- Este apartado busca examinar la estructura y composición de los datos, con el fin de ejecutar una serie de transformaciones que brinden estabilidad para el manejo y adaptación de los datos.

In [145]:
df

,Fecha,Euro - USD/EUR - Tasa media(Dato fin de semana),Dólar australiano - COP/AUD - Tasa media(Dato fin de semana),Dólar canadiense - COP/CAD - Tasa media(Dato fin de semana),Euro - COP/EUR - Tasa media(Dato fin de semana)
0,01/01/2009,-,-,-,-
1,02/01/2009,-,-,-,-
2,03/01/2009,"1,39340","1.577,10541","1.838,89574","3.113,98425"
3,04/01/2009,"1,39340","1.577,10541","1.838,89574","3.113,98425"
4,05/01/2009,-,-,-,-
...,...,...,...,...,...
6056,03/08/2025,"1,15545","2.668,22473","2.994,61184","4.771,33833"
6057,04/08/2025,-,-,-,-
6058,05/08/2025,-,-,-,-
6059,06/08/2025,-,-,-,-


**Observaciones**
- Se utiliza el "-" como señalizador de que no existe ningún valor
- Los números utilizan como separador de miles "." y para números decimales ",".
- La ausencia de valores se debe a la fecha, debido a que cada valor depende del tiempo establecido de la recolección del dato.
- Existen registros duplicados para los días que corresponden al fin de semana.
- Las últimos 4 registros no poseen datos al no ser fechas de fin de semana.

In [146]:
rows, columns = df.shape
print(f"El tamaño del Dataframe es de {rows} filas x {columns}")

El tamaño del Dataframe es de 6061 filas x 5


In [147]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6061 entries, 0 to 6060
Data columns (total 5 columns):
 #   Column                                                        Non-Null Count  Dtype 
---  ------                                                        --------------  ----- 
 0   Fecha                                                         6061 non-null   object
 1   Euro - USD/EUR - Tasa media(Dato fin de semana)               6061 non-null   object
 2   Dólar australiano - COP/AUD - Tasa media(Dato fin de semana)  6061 non-null   object
 3   Dólar canadiense - COP/CAD - Tasa media(Dato fin de semana)   6061 non-null   object
 4   Euro - COP/EUR - Tasa media(Dato fin de semana)               6061 non-null   object
dtypes: object(5)
memory usage: 236.9+ KB


**Observaciones**
- Las columnas son de tipo objeto (cadena de texto), lo que implica una conversión de tipo a númerico decimal.
- La columna de Fecha Requiere segmentación en día, mes y año, para un mejor manejo de los datos.

In [148]:
for column in df.columns:
    print(f"Existen {df[column].isna().sum()} en la columna {column}")

Existen 0 en la columna Fecha
Existen 0 en la columna Euro - USD/EUR - Tasa media(Dato fin de semana)
Existen 0 en la columna Dólar australiano - COP/AUD - Tasa media(Dato fin de semana)
Existen 0 en la columna Dólar canadiense - COP/CAD - Tasa media(Dato fin de semana)
Existen 0 en la columna Euro - COP/EUR - Tasa media(Dato fin de semana)


**Observaciones**
- No haber valores faltantes (nan) diferentes a los denotados con "-".

In [149]:
df['Fecha'].unique()

array(['01/01/2009', '02/01/2009', '03/01/2009', ..., '05/08/2025',
       '06/08/2025', '07/08/2025'], dtype=object)

**Observaciones**
- Existen valores no congruentes con el formato del dataframe (ej:'Descargado de sistema del Banco de la República jueves, 7 de agosto de 2025 2:02:36 p.\xa0m.)

## 1.3 Transformación de Datos

**Acciones a Realizar**
1. Separar el campo Fecha en 3 columnas (día, mes y año)
2. Renombrar las columnas 
3. Eliminación de registros irrelevantes (no aportan información)
4. Transformación de Tipos 
5. Eliminación de Información Duplicada
6. Conversiones de divisas (USD a COP)


### **1. Separar el campo Fecha en 3 columnas (día, mes y año)**

In [150]:
date_dict = {
    "Day": [],
    "Month": [],
    "Year": []
}

def divide_date(date : str):
    '''
        Extrae de cada registro de la columnna 'Fecha' 
        el día, mes y año
        
        Parámetro:
        - date: registro de 'Fecha'

        Retorna:
        - date: registro de fecha
    '''

    if pd.isna(date):
        return date
    
    if '/' not in date:
        return date

    date_parts = date.split("/")

    date_dict["Day"].append(date_parts[0])
    date_dict["Month"].append(date_parts[1])
    date_dict["Year"].append(date_parts[2])

    return date

# Se llena el diccionario con los valores seccionados
df['Fecha'].apply(divide_date)
# Se unen el diccionario al dataset
df = pd.concat([df, pd.DataFrame(date_dict)],axis=1)
# Se borra la columna Fecha
#df = df.drop(columns=['Fecha'])

df.head()

,Fecha,Euro - USD/EUR - Tasa media(Dato fin de semana),Dólar australiano - COP/AUD - Tasa media(Dato fin de semana),Dólar canadiense - COP/CAD - Tasa media(Dato fin de semana),Euro - COP/EUR - Tasa media(Dato fin de semana),Day,Month,Year
0,01/01/2009,-,-,-,-,01,01,2009
1,02/01/2009,-,-,-,-,02,01,2009
2,03/01/2009,"1,39340","1.577,10541","1.838,89574","3.113,98425",03,01,2009
3,04/01/2009,"1,39340","1.577,10541","1.838,89574","3.113,98425",04,01,2009
4,05/01/2009,-,-,-,-,05,01,2009


### **2. Renombramiento de columnas**

**Convención a Utilizar**
- Con el fin de simplificar los nombres del dataset, se agregará la palabra COP, seguido de la divisa, de la siguiente forma: COP_DIVISA
- Debido a que se tienen distintas columnas en base a datos temporales, se establecerá una convención de la siguiente forma:
  - Semana -> W (ej: COP_EURO_W)
  - Semestre -> S (ej: COP_USD_S)
- Dado que los dólares estadounidenses no se encontraron directamente convertidos a pesos colombianos, se denotará de forma similar pero con USD, aunque posteriormente se hará su respectiva conversión a pesos colombianos.

In [ ]:
new_name_dict = {
    "Euro - USD/EUR - Tasa media(Dato fin de semana)": "USD_EUR_W",
    "Dólar australiano - COP/AUD - Tasa media(Dato fin de semana)":"AUD",
    "Dólar canadiense - COP/CAD - Tasa media(Dato fin de semana)":"CAD",
    "Euro - COP/EUR - Tasa media(Dato fin de semana)":"EUR",
}
# Se pasa el diccionario, donde la clave es el nombre antiguo y el valor el nuevo nombre
df = df.rename(new_name_dict,axis=1)
df.head()

,Fecha,USD_EUR_W,AUD,CAD,EUR,Day,Month,Year
0,01/01/2009,-,-,-,-,01,01,2009
1,02/01/2009,-,-,-,-,02,01,2009
2,03/01/2009,"1,39340","1.577,10541","1.838,89574","3.113,98425",03,01,2009
3,04/01/2009,"1,39340","1.577,10541","1.838,89574","3.113,98425",04,01,2009
4,05/01/2009,-,-,-,-,05,01,2009


### **3. Eliminación de registros irrelevantes (no aportan información)**
- Dada la existencia de registros donde no existen valores para los precios al no ser fechas de fin de semana, se procede a eliminarlos del dataset.

In [152]:
# Extrar registros con información válida
df = df[(df['USD_EUR_W'] != "-") & (df['AUD'] != "-") & (df['CAD'] != "-") & (df['EUR'] != "-")]
# Reset del índice
df =df.reset_index(drop=True)

df

,Fecha,USD_EUR_W,AUD,CAD,EUR,Day,Month,Year
0,03/01/2009,"1,39340","1.577,10541","1.838,89574","3.113,98425",03,01,2009
1,04/01/2009,"1,39340","1.577,10541","1.838,89574","3.113,98425",04,01,2009
2,10/01/2009,"1,35090","1.561,66646","1.858,16215","2.993,90510",10,01,2009
3,11/01/2009,"1,35090","1.561,66646","1.858,16215","2.993,90510",11,01,2009
4,17/01/2009,"1,32510","1.496,88957","1.784,71398","2.951,89876",17,01,2009
...,...,...,...,...,...,...,...,...
1725,20/07/2025,"1,16475","2.610,19799","2.916,21532","4.659,71049",20,07,2025
1726,26/07/2025,"1,17305","2.696,26935","2.999,58404","4.821,79856",26,07,2025
1727,27/07/2025,"1,17305","2.696,26935","2.999,58404","4.821,79856",27,07,2025
1728,02/08/2025,"1,15545","2.668,22473","2.994,61184","4.771,33833",02,08,2025


### **4. Transformación de Tipos**
- Se procede a quitar el separador de miles "." y colocar el indicador de decimales "." en vez de ","
- En los campos donde no existe ningun valor, se colocará un valor por defecto de NaN perteneciente a NumPy (np.nan).
- Se transformará la columna a númerico decimal (float) o entero, dependiendo si es una columna de carácter temporal o monetaria.

In [153]:
integer_columns = ['Day','Month','Year']

for column in df.columns:
    # Remplazo de separadores
    df[column] = df[column].str.replace(".","",regex=False)
    df[column] = df[column].str.replace(",",".",regex=False)
    
    # Conversión a tipos dependiendo del tipo de columna (Temporal o Monetaria)

    if column == 'Fecha':
        continue
    if column not in integer_columns:
        df[column] = df[column].astype(float)
    else:
        df[column] = df[column].astype(int)

df.head()

,Fecha,USD_EUR_W,AUD,CAD,EUR,Day,Month,Year
0,03/01/2009,1.3934,1577.10541,1838.89574,3113.98425,3,1,2009
1,04/01/2009,1.3934,1577.10541,1838.89574,3113.98425,4,1,2009
2,10/01/2009,1.3509,1561.66646,1858.16215,2993.90510,10,1,2009
3,11/01/2009,1.3509,1561.66646,1858.16215,2993.90510,11,1,2009
4,17/01/2009,1.3251,1496.88957,1784.71398,2951.89876,17,1,2009


### **5. Eliminación de Información Duplicada**
- Dado que la base de datos presenta información duplicada en el fin de semana, se prosigue a eliminar los registros duplicados (aquellos cuyo dia = dia+1 y los valores son los mismos).

In [154]:
# Se crea una columna que indique si se repite el registro 
df['repeated'] = (
    (df['Day'] == df['Day'].shift(1)+1) & 
    (df['Month'] == df['Month'].shift(1)) & 
    (df['Year'] == df['Year'].shift(1)) & 
    (df['EUR'] == df['EUR'].shift(1)))

# Se filtran las que no están repetidas
df = df.where(df['repeated'] == False)

# Se borran los registros NaN (que aparecen en vez de las filas repetidas)
df = df.dropna()

df = df.drop(columns=['repeated'])

df.head()

,Fecha,USD_EUR_W,AUD,CAD,EUR,Day,Month,Year
0,03/01/2009,1.39340,1577.10541,1838.89574,3113.98425,3.0,1.0,2009.0
2,10/01/2009,1.35090,1561.66646,1858.16215,2993.90510,10.0,1.0,2009.0
4,17/01/2009,1.32510,1496.88957,1784.71398,2951.89876,17.0,1.0,2009.0
6,24/01/2009,1.27905,1484.32511,1825.63835,2917.21886,24.0,1.0,2009.0
8,31/01/2009,1.28150,1539.04333,1952.05871,3101.56319,31.0,1.0,2009.0


- Teniendo en cuente el factor de que el drop.na() cambio el formato de las columnas, se prosigue a restaurarlo antes de continuar.

In [155]:
integer_columns = ['Day','Month','Year']

for column in df.columns:

    if column == 'Fecha':
        continue
    # Conversión a tipos dependiendo del tipo de columna (Temporal o Monetaria)
    if column not in integer_columns:
        df[column] = df[column].astype(float)
    else:
        df[column] = df[column].astype(int)

df = df.reset_index(drop=True)
df.head()

,Fecha,USD_EUR_W,AUD,CAD,EUR,Day,Month,Year
0,03/01/2009,1.39340,1577.10541,1838.89574,3113.98425,3,1,2009
1,10/01/2009,1.35090,1561.66646,1858.16215,2993.90510,10,1,2009
2,17/01/2009,1.32510,1496.88957,1784.71398,2951.89876,17,1,2009
3,24/01/2009,1.27905,1484.32511,1825.63835,2917.21886,24,1,2009
4,31/01/2009,1.28150,1539.04333,1952.05871,3101.56319,31,1,2009


### **6. Conversiones de divisas (USD a COP)**


- A pesar de que la base de datos adquirida del Banco de la República no tenía el trayecto de dólares estadounidenses a pesos colombianos, se pudo calcular a partir del historial de dólares estadounidenses a euros.

$$
COP_{USD} = \frac{1}{USD_{EUR}} \cdot COP_{EUR}
$$


In [156]:
# Se calculan los dólares a pesos colombianos
df['USD'] = (1/df['USD_EUR_W'])*df['EUR']

# Se redondean los números a 5 dígitos de aproximación
df['USD'] = round(df['USD'],5)

# Se elimina la columna de dólares a euros (ya no es necesaria)
df = df.drop(columns=['USD_EUR_W'])

df

,Fecha,AUD,CAD,EUR,Day,Month,Year,USD
0,03/01/2009,1577.10541,1838.89574,3113.98425,3,1,2009,2234.81000
1,10/01/2009,1561.66646,1858.16215,2993.90510,10,1,2009,2216.22999
2,17/01/2009,1496.88957,1784.71398,2951.89876,17,1,2009,2227.67999
3,24/01/2009,1484.32511,1825.63835,2917.21886,24,1,2009,2280.76999
4,31/01/2009,1539.04333,1952.05871,3101.56319,31,1,2009,2420.26000
...,...,...,...,...,...,...,...,...
890,05/07/2025,2603.01363,2920.18368,4681.21170,5,7,2025,3974.37000
891,12/07/2025,2631.96366,2923.30157,4681.96046,12,7,2025,4003.90000
892,19/07/2025,2610.19799,2916.21532,4659.71049,19,7,2025,4000.60999
893,26/07/2025,2696.26935,2999.58404,4821.79856,26,7,2025,4110.48000


# **3. Definición Variable de Estado y Estados del Mercado**

Con respecto a la variable de estado, se definirá de la siguiente forma:
$$
X_t := \text{El comportamiento del euro y el dólar estadounidense, canadiense y australiano en la semana } t, \ t=1,2,3,\dots
$$


En lo que concierne a los estados, se definirán de la siguiente forma:
- Se calculará el cambio de la divisa con respecto al registro anterior, donde si hubo un incremento mayor al 5% se considerará como SUBIO (S), si se presento un decremento menor al 5% se clasificará como BAJO (B). Finalmente, si no supero los límites del 5% en incremento y decremento, se clasificará como ESTABLE (E).
- Los estados abarcarán las 4 divisas con la información previa, donde cada moneda irá acompañada del cambio (S,B,E).
- La simplificación del nombre de las divisas segfuirá el siguiente orden: 
  - AUD : (A)
  - CAD : (C)
  - EUR : (R)
  - USD : (U)
- Ejemplo: AE CB RS US -> (AUD se mantuvo estable, CAD bajo, EUR SUBIO, USD SUBIÓ)
- De esta manera existen los siguientes estados:


In [ ]:
columns = ['CAD','AUD','USD','EUR']

treshold = 0.005

for col in columns:
    cambio = df[col].pct_change()  # cambio porcentual respecto al anterior

    currency = col[0] if col[0] != "E" else 'R'
    df[f"{currency}_STATUS"] = np.where(
        cambio > treshold, 'S',
        np.where(cambio < -treshold, 'B', 'E')
    )

df.head()

,Fecha,AUD,CAD,EUR,Day,Month,Year,USD,C_STATUS,A_STATUS,U_STATUS,R_STATUS
0,03/01/2009,1577.10541,1838.89574,3113.98425,3,1,2009,2234.81000,E,E,E,E
1,10/01/2009,1561.66646,1858.16215,2993.90510,10,1,2009,2216.22999,E,E,E,B
2,17/01/2009,1496.88957,1784.71398,2951.89876,17,1,2009,2227.67999,B,B,E,E
3,24/01/2009,1484.32511,1825.63835,2917.21886,24,1,2009,2280.76999,S,E,S,E
4,31/01/2009,1539.04333,1952.05871,3101.56319,31,1,2009,2420.26000,S,S,S,S


In [158]:
columns = ['C_STATUS','A_STATUS','U_STATUS','R_STATUS']

df['STATUS'] = ""

for column in columns:
    df[column] = column[0]+df[column]+" "

    df['STATUS'] = df['STATUS']+df[column]


df.head()

,Fecha,AUD,CAD,EUR,Day,Month,Year,USD,C_STATUS,A_STATUS,U_STATUS,R_STATUS,STATUS
0,03/01/2009,1577.10541,1838.89574,3113.98425,3,1,2009,2234.81000,CE,AE,UE,RE,CE AE UE RE
1,10/01/2009,1561.66646,1858.16215,2993.90510,10,1,2009,2216.22999,CE,AE,UE,RB,CE AE UE RB
2,17/01/2009,1496.88957,1784.71398,2951.89876,17,1,2009,2227.67999,CB,AB,UE,RE,CB AB UE RE
3,24/01/2009,1484.32511,1825.63835,2917.21886,24,1,2009,2280.76999,CS,AE,US,RE,CS AE US RE
4,31/01/2009,1539.04333,1952.05871,3101.56319,31,1,2009,2420.26000,CS,AS,US,RS,CS AS US RS


# **4. Construcción del Modelo de la Cadena de Markov**

In [159]:
auxiliar_table = df.copy()
auxiliar_table = auxiliar_table.drop(columns=['AUD','CAD','EUR','USD','C_STATUS','A_STATUS','U_STATUS','R_STATUS'])

auxiliar_table = auxiliar_table.rename(columns={
    'STATUS':'X_t',
    'Fecha':'t'})


auxiliar_table.head()

,t,Day,Month,Year,X_t
0,03/01/2009,3,1,2009,CE AE UE RE
1,10/01/2009,10,1,2009,CE AE UE RB
2,17/01/2009,17,1,2009,CB AB UE RE
3,24/01/2009,24,1,2009,CS AE US RE
4,31/01/2009,31,1,2009,CS AS US RS


In [164]:
auxiliar_table['X_t'].value_counts()


X_t
CE AE UE RE     393
CS AS US RS      58
CB AB UB RB      45
CE AE UE RB      29
CE AE US RE      27
CE AB UE RE      27
CE AE UB RE      26
CE AS UE RE      26
CE AE UE RS      24
CE AE US RS      20
CE AE UB RB      20
CS AE US RS      18
CB AE UB RB      17
CB AB UE RE      16
CE AB UE RB      14
CB AE UE RE      11
CS AS UE RS      11
CS AE US RE      10
CE AB UB RB      10
CB AB UB RE       9
CB AB UE RB       9
CS AE UE RE       9
CS AS US RE       8
CE AS UE RS       8
CE AS US RS       8
CS AS UE RE       7
CB AE UE RB       6
CS AE UE RS       5
CE AS US RE       4
CE AB UB RE       4
CB AE UB RE       3
CE AS UB RE       2
CS AS UE RB       1
CB AS UB RB       1
CE AB US RE       1
CE AB US RS       1
CE AB UE RS       1
CS AE US RB       1
CE AS UE RB       1
CB AE UE RS       1
CB AB UE RS       1
CS AE UB RE       1
CS AB US RS       1
Name: count, dtype: int64